<h1>Merge DataFrame together to useful Dataformat</h1>

<h2>Imports and makros</h2>

In [3]:
import pandas as pd
%matplotlib widget

potential function arguments

In [17]:
participant_number = 1
questionaire_path = "./rawData/Questionnaire.xlsx"

In [23]:
df_questionaire = pd.read_excel(questionaire_path)
df_questionaire = df_questionaire.transpose()
df_questionaire.columns = df_questionaire.iloc[0]
df_questionaire = df_questionaire.iloc[1:]
display(df_questionaire)
df_questionaire = df_questionaire.iloc[participant_number]
df_questionaire

Frage\Participant:,age,sex,hand,colorblindness,concentrationDifficulty,readingDifficulty,job,education
1,27,male,right,no,1,1,Universitätsmitarbeiter,Master
2,26,female,right,no,1,4,StudentMaster,Bachelor
3,26,male,right,no,1,1,StudentMaster,Bachelor
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,26,male,right,no,1,1,ProfessionalSoftwareengineering,BeruflicheAusbildung


Frage\Participant:
age                                   26
sex                               female
hand                               right
colorblindness                        no
concentrationDifficulty                1
readingDifficulty                      4
job                        StudentMaster
education                       Bachelor
Name: 2, dtype: object